In [10]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

import string
import datetime
import time

In [29]:
# global variables
%store -r playersToIds
%store -r idsToPlayers
%store -r datesDict
%store -r statsNBAMap

In [103]:
######## FIND PLUS MINUS OF TEAMS PLAYER HAS PLAYED FOR, INCLUDING REST DAY PLUS MINUS ##########
def getTeams(playerName):
    playerId = playersToIds[playerName]
    url = getMainPlayerURL(playerId)
    
    html = urlopen(url)
    soup = BeautifulSoup(html)
    seasons = soup.findAll('tr')
    teams = []
    for i in range(1, len(seasons)):
        season = seasons[i]
        if season.th:
            if season.th.getText() == "Career": # career table starts
                break
        else: # did not play this season
            continue
        yearStr = season['id']
        colon = yearStr.index('.')
        year = yearStr[colon+1:]
        
        seasonYear = str(int(year) - 1) + "-" + year[2:]
        
        teamName = season.findAll('td')[1].getText()
        
        if teamName == "TOT":
            continue
        
        advUrl = "https://stats.nba.com/team/{}/advanced/?Season={}&SeasonType=Regular%20Season"
        advUrl = advUrl.format(statsNBAMap[teamName], seasonYear)
        
        options = setupHeadlessChrome()
            
        driver = webdriver.Chrome() # used to view generated JavaScript
        driver.get(advUrl)

        html = driver.page_source
        table = driver.find_elements_by_tag_name('tbody')
        while len(table) == 0:
            table = driver.find_elements_by_tag_name('tbody')
        a = table[0].find_elements_by_tag_name('td')
        
        netRatings = {}
        netrtg = a[5].get_attribute('innerHTML')
        print("Team Net Rating: ", netrtg)
        netRatings['Team'] = netrtg
        
        z = table[-2].find_elements_by_tag_name('tr')
        
        for i in range(len(z)):
            
            z1 = z[i].find_elements_by_tag_name('td')
            day = z1[0].text
            rtg = z1[5].get_attribute('innerHTML')
            netRatings[day] = rtg
        
        driver.close()
        print(netRatings)
        
    return teams

######## Parse Basketball Reference URL for Player's Main Page ##########
def getMainPlayerURL(playerId):
    return "https://www.basketball-reference.com/players/{}/{}.html".format(playerId[0], playerId)

# Helper for team exceptions, some teams have inconsistent team IDs
def streakFindingTeam(team):
    if team == 'BRK':
        return 'NJN'
    else:
        return team

def setupHeadlessChrome():
    options = webdriver.ChromeOptions() # don't open URL window
    options.add_argument('--headless')
    prefs = {"profile.managed_default_content_settings.images": 2, 'disk-cache-size': 4096 }
    options.add_experimental_option("prefs", prefs)
    return options

In [102]:
getTeams('LeBron James')

2003-04
0
Team Net Rating:  -2.6
{'Team': '-2.6', '0 Days Rest': '-2.8', '1 Days Rest': '-1.3', '2 Days Rest': '-5.0', '3 Days Rest': '-0.3', '6+ Days Rest': '-15.6'}
2004-05
12
Team Net Rating:  0.6
{'Team': '0.6', '0 Days Rest': '-4.8', '1 Days Rest': '-0.7', '2 Days Rest': '9.8', '3 Days Rest': '8.9', '4 Days Rest': '13.4', '5 Days Rest': '7.3', '6+ Days Rest': '-3.8'}
2005-06
12
Team Net Rating:  2.1
{'Team': '2.1', '0 Days Rest': '-2.4', '1 Days Rest': '0.4', '2 Days Rest': '8.6', '3 Days Rest': '9.1', '5 Days Rest': '15.3', '6+ Days Rest': '22.0'}
2006-07
0
Team Net Rating:  3.8
{'Team': '3.8', '0 Days Rest': '-0.5', '1 Days Rest': '5.9', '2 Days Rest': '4.2', '3 Days Rest': '6.8', '5 Days Rest': '1.1', '6+ Days Rest': '2.1'}
2007-08
0
Team Net Rating:  -0.6
{'Team': '-0.6', '0 Days Rest': '-5.3', '1 Days Rest': '1.1', '2 Days Rest': '0.9', '3 Days Rest': '6.1', '5 Days Rest': '-14.8'}
2008-09
0
Team Net Rating:  9.6
{'Team': '9.6', '0 Days Rest': '8.1', '1 Days Rest': '8.7', '2 

[]